# 0. Libraries

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import date
today = date.today()
path = os.path.dirname(os.getcwd())
print(f'📂 Current working directory: {path}')
print(f'💚 Today is {today}')
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'scripts'))
import ss_api_call as ss

📂 Current working directory: /Users/serenekim/Desktop/PhD/meta-wealth_mobility
💚 Today is 2025-09-18


# 1. Triangles

In [2]:
tri = pd.read_csv(f"{path}/results/feature-only-KG/triangle_counts_papers.csv")
len(tri)

495

In [4]:
import re, unicodedata
import pandas as pd

# --- 1) Normalizer: unify Unicode dashes to "-" and collapse spaces ---
def norm(s):
    if pd.isna(s): return s
    s = unicodedata.normalize("NFKC", str(s))
    s = re.sub(r"[\u2010\u2011\u2012\u2013\u2014\u2212]", "-", s)  # dash variants → "-"
    s = re.sub(r"\s+", " ", s).strip()
    return s

for base in ("m","dt","rq"):
    tri[f"{base}.name_norm"] = tri[f"{base}.name"].map(norm)

# --- 2) Your label → code dictionaries (raw), then normalized once ---
M_MAP = {
    "Regression‐based Measures": 1,
    "Rank‐based Measures": 2,
    "Transition Matrix / Probability Measures": 3,
    "Absolute Mobility Measures": 4,
    "Multigenerational Measures": 5,
    "Decomposition / Structural Approaches": 6,
    "Non-parametric Approaches": 7,
    "Others_Measure": 8,
}
DT_MAP = {
    "Panel/Longitudinal Surveys": 1,
    "Administrative/Registry Data": 2,
    "National Survey Data": 3,
    "Opportunity Atlas": 4,
    "Natural/Experimental Data": 5,
    "Linked Administrative Data": 6,
    "International Panel Data": 7,
    "Rich List Data": 8,
    "University/Institution Data": 9,
    "Pseudo-Panel/Household Budget Survey": 10,
    "Archival/Historical Data": 11,
    "Big Data": 12,
    "No dataset": 13,
    "Others_DataType": 14,
}
RQ_MAP = {
    "Measurement and Methodological Advances": 1,
    "Empirical Estimates and Determinants": 2,
    "Policy, Institutional, and Geographic Impacts": 3,
    "Intergenerational Wealth Mobility and Inheritance": 4,
    "Demographic Differences in Mobility (Race, Gender, etc.)": 5,
    "Mobility and Non-Income Outcomes (Health, Wellbeing, etc.)": 6,
    "Theoretical and Structural Models": 7,
    "Perceptions of Mobility and Attitudes": 8,
    "Others_RqType": 9,
}

M_MAP = {norm(k): v for k,v in M_MAP.items()}
DT_MAP = {norm(k): v for k,v in DT_MAP.items()}
RQ_MAP = {norm(k): v for k,v in RQ_MAP.items()}

# --- 3) Vectorized mapping (nullable Int64 keeps NA if truly unseen) ---
tri["m.code"]  = tri["m.name_norm"].map(M_MAP).astype("Int64")
tri["dt.code"] = tri["dt.name_norm"].map(DT_MAP).astype("Int64")
tri["rq.code"] = tri["rq.name_norm"].map(RQ_MAP).astype("Int64")

tri["code_combined"] = tri[['m.code', 'dt.code', 'rq.code']].apply(lambda x: [v for v in x if pd.notna(v)], axis=1)
tri["code_combined_flt"] = tri[['m.code', 'dt.code', 'rq.code']].apply(lambda x: [v for v in x if pd.notna(v)], axis=1)
tri['code_combined'] = tri['code_combined'].astype('string')
# (optional) quick diagnostics for any still-unmapped labels
for base in ("m","dt","rq"):
    miss = tri.loc[tri[f"{base}.code"].isna(), f"{base}.name"].unique()
    if len(miss): print(f"Unmapped {base}.name values:", miss)


In [5]:
tri['names'].nunique(), tri['code_combined'].nunique() 

(227, 227)

In [6]:
tri['period2'] = tri['period'].apply(lambda x: x.split('-')[1].strip())
export = tri[['period2', 'names', 'n_papers', 'code_combined']].copy()
# export.to_csv(f"{path}/results/feature-only-KG/triangle_counts_papers_p2.csv", index=False)
export

,period2,names,n_papers,code_combined
0,2000,"[No dataset, Decomposition / Structural Approa...",4,"[6, 13, 7]"
1,2000,"[Administrative/Registry Data, Regression‐base...",2,"[1, 2, 4]"
2,2000,"[Linked Administrative Data, Regression‐based ...",2,"[1, 6, 4]"
3,2000,"[No dataset, Regression‐based Measures, Empiri...",2,"[1, 13, 2]"
4,2000,"[Panel/Longitudinal Surveys, Regression‐based ...",2,"[1, 1, 2]"
...,...,...,...,...
490,2025,"[Panel/Longitudinal Surveys, Regression‐based ...",0,"[1, 1, 6]"
491,2025,"[Opportunity Atlas, Regression‐based Measures,...",0,"[1, 4, 6]"
492,2025,"[Others_DataType, Rank‐based Measures, Mobilit...",0,"[2, 14, 6]"
493,2025,"[Panel/Longitudinal Surveys, Rank‐based Measur...",0,"[2, 1, 6]"


In [7]:
first = export.groupby('period2').first()
np.column_stack([first.index, first['names'], first['code_combined']])  # returns a numpy array with period2 and names

array([['2000',
        '[No dataset, Decomposition / Structural Approaches, Theoretical and Structural Models]',
        '[6, 13, 7]'],
       ['2005',
        '[Panel/Longitudinal Surveys, Regression‐based Measures, Intergenerational Wealth Mobility and Inheritance]',
        '[1, 1, 4]'],
       ['2010',
        '[Panel/Longitudinal Surveys, Regression‐based Measures, Intergenerational Wealth Mobility and Inheritance]',
        '[1, 1, 4]'],
       ['2015',
        '[Panel/Longitudinal Surveys, Regression‐based Measures, Intergenerational Wealth Mobility and Inheritance]',
        '[1, 1, 4]'],
       ['2020',
        '[Panel/Longitudinal Surveys, Regression‐based Measures, Empirical Estimates and Determinants]',
        '[1, 1, 2]'],
       ['2025',
        '[Panel/Longitudinal Surveys, Regression‐based Measures, Empirical Estimates and Determinants]',
        '[1, 1, 2]']], dtype=object)

In [17]:
export.groupby(['names', 'code_combined']).sum('n_papers').sort_values('n_papers', ascending=False).head(10)

,,n_papers
names,code_combined,
"[Panel/Longitudinal Surveys, Regression‐based Measures, Empirical Estimates and Determinants]","[1, 1, 2]",47
"[Panel/Longitudinal Surveys, Regression‐based Measures, Intergenerational Wealth Mobility and Inheritance]","[1, 1, 4]",47
"[Linked Administrative Data, Regression‐based Measures, Intergenerational Wealth Mobility and Inheritance]","[1, 6, 4]",30
"[National Survey Data, Regression‐based Measures, Empirical Estimates and Determinants]","[1, 3, 2]",24
"[National Survey Data, Regression‐based Measures, Intergenerational Wealth Mobility and Inheritance]","[1, 3, 4]",23
"[No dataset, Decomposition / Structural Approaches, Theoretical and Structural Models]","[6, 13, 7]",21
"[No dataset, Regression‐based Measures, Empirical Estimates and Determinants]","[1, 13, 2]",19
"[Administrative/Registry Data, Regression‐based Measures, Intergenerational Wealth Mobility and Inheritance]","[1, 2, 4]",15
"[Administrative/Registry Data, Regression‐based Measures, Empirical Estimates and Determinants]","[1, 2, 2]",13


## Beeswarm plot with Plotly (I don't like it)

In [9]:
# df = tri[tri['n_papers']>0].copy()
df = tri.copy()
len(df)

495

In [10]:
import plotly.express as px

# Columns based on your request
x_col = "period2" if "period2" in df.columns else "period"
size_col = "n_papers" if "n_papers" in df.columns else None
color_col = "names" 

# color_profile = {
#     '[1,1,4]':
#     '[1,1,2]':
# }

# Ensure numeric x
df[x_col] = pd.to_numeric(df[x_col], errors="coerce")
df = df[df[x_col].notna()].copy()

# --- Circular disk layout per x bucket ---
# Parameters to control the circle size in data units
rx = 0.4  # horizontal radius in "year" units
ry = 1.0  # vertical radius in arbitrary y units

def disk_positions(n, rx=0.4, ry=1.0):
    """
    Arrange n points in concentric hex-like rings that fill a disk.
    Returns arrays of x_offset, y_offset centered at (0,0).
    Ring k has capacity 6*k (k>=1), ring 0 has 1 at center.
    """
    xs, ys = [], []
    if n <= 0:
        return np.array([]), np.array([])
    # center
    xs.append(0.0)
    ys.append(0.0)
    placed = 1
    k = 2
    while placed < n:
        cap = 6 * k
        thetas = np.linspace(0, 2*np.pi, cap, endpoint=False)
        for th in thetas:
            if placed >= n:
                break
            xs.append(rx * k * np.cos(th))
            ys.append(ry * k * np.sin(th))
            placed += 1
        k += 1
    return np.array(xs), np.array(ys)

# Within each x group, optionally sort by size descending so large dots gravitate toward center
if size_col and size_col in df.columns:
    df["_size_for_sort"] = df[size_col].fillna(1.0)
else:
    df["_size_for_sort"] = 1.0

df["_x"] = df[x_col]
df["_y"] = 0.0

for xval, idxs in df.groupby(x_col).groups.items():
    ids = list(idxs)
    # sort big -> small
    ids_sorted = df.loc[ids].sort_values("_size_for_sort", ascending=False).index.tolist()
    nx = len(ids_sorted)
    xs_off, ys_off = disk_positions(nx, rx=rx, ry=ry)
    # assign offsets in sorted order
    for i, row_id in enumerate(ids_sorted):
        df.at[row_id, "_x"] = xval + xs_off[i]
        df.at[row_id, "_y"] = ys_off[i]

# --- Plot ---
kwargs = {}
if size_col and size_col in df.columns:
    kwargs.update(size=size_col, size_max=30)

fig = px.scatter(
    df,
    x="_x",
    y="_y",
    color=color_col,
    hover_data=[x_col, color_col] + ([size_col] if size_col in df.columns else []),
    **kwargs
)

# Cosmetics
fig.update_traces(marker=dict(opacity=0.85), selector=dict(mode="markers"))
fig.update_yaxes(showgrid=False, showticklabels=False, zeroline=True, zerolinewidth=1)
fig.update_xaxes(title=x_col)
fig.update_layout(
    title=f"Circular beeswarm per {x_col} (size = {size_col or 'constant'}, color = {color_col})",
    # legend_title=color_col,
    showlegend=False,
    height=560,
    margin=dict(l=40, r=20, t=60, b=40),
)

fig.show()


/var/folders/xy/r3gq5vtd7bx6qb966l0fhh9h0000gn/T/ipykernel_4069/2569920093.py:65: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2000.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[row_id, "_x"] = xval + xs_off[i]


In [86]:
fig.write_html(f"{path}/results/feature-only-KG/triangle_counts_papers_circular_beeswarm.html", include_plotlyjs="cdn")

# 2. Missing triangles per year

In [23]:
# --- 1) Clean types and build a canonical tuple key --------------------------
# If your codes are already integers, you can skip the astype lines.
tri = tri.copy()
tri['m.code']  = pd.to_numeric(tri['m.code'], errors='coerce').astype('Int64')
tri['dt.code'] = pd.to_numeric(tri['dt.code'], errors='coerce').astype('Int64')
tri['rq.code'] = pd.to_numeric(tri['rq.code'], errors='coerce').astype('Int64')

# keep only fully-specified triangles
tri_ok = tri.dropna(subset=['m.code','dt.code','rq.code']).copy()
tri_ok['triple'] = list(zip(tri_ok['m.code'].astype(int),
                            tri_ok['dt.code'].astype(int),
                            tri_ok['rq.code'].astype(int)))

# --- 2) Universe of possible combinations -----------------------------------
n_measures = range(1, 9)    
n_dt       = range(1, 15)   
n_rq       = range(1, 10)  

all_possible = {(m, d, r) for m in n_measures for d in n_dt for r in n_rq}
appeared_all = set(tri_ok['triple'])

# --- 3) Focus case: m=1, d=1 -------------------------------------------------
M, D = 1, 2
Rs_all_possible = {r for (m, d, r) in all_possible if m == M and d == D}
Rs_seen_overall = {r for (m, d, r) in appeared_all  if m == M and d == D}

# (A) r's that SHOULD exist (by universe) but NEVER appear in your data at all
missing_overall_rs = sorted(Rs_all_possible - Rs_seen_overall)

missing_overall_df = pd.DataFrame(
    [(M, D, r) for r in missing_overall_rs],
    columns=['m.code','dt.code','rq.code']
)

# --- 4) Period-wise: missing in period A but present in other periods --------
# Assume periods are in tri_ok['period'] (string like '-2000', '2001-2005', …)
# If your column is named differently, change below.
if 'period' not in tri_ok.columns:
    raise KeyError("Expected a 'period' column in tri.")

# r-values per period for m=1,d=1
per = tri_ok.loc[(tri_ok['m.code']==M)&(tri_ok['dt.code']==D), ['rq.code','period']].copy()
per['rq.code'] = per['rq.code'].astype(int)

rs_by_period = per.groupby('period')['rq.code'].agg(lambda s: set(s.tolist())).to_dict()
all_periods  = list(rs_by_period.keys())

rows = []
for A in all_periods:
    rs_in_A      = rs_by_period.get(A, set())
    rs_elsewhere = set().union(*(rs_by_period[p] for p in all_periods if p != A))
    # r that appear somewhere else (same m,d) but NOT in A
    missing_in_A_but_seen_elsewhere = sorted((rs_elsewhere - rs_in_A) & Rs_all_possible)
    # also useful: r that are missing in A relative to the universe (whether or not seen elsewhere)
    missing_in_A_vs_universe        = sorted(Rs_all_possible - rs_in_A)
    rows.append({
        'period': A,
        'missing_in_A_but_seen_elsewhere': missing_in_A_but_seen_elsewhere,
        'missing_in_A_vs_universe': missing_in_A_vs_universe,
        'appeared_in_A': sorted(rs_in_A),
    })

missing_by_period_df = pd.DataFrame(rows).sort_values('period').reset_index(drop=True)

In [24]:
missing_by_period_df # Focusing on (M,D) = (1,1)

,period,missing_in_A_but_seen_elsewhere,missing_in_A_vs_universe,appeared_in_A
0,-2000,"[1, 2, 3, 5]","[1, 2, 3, 5, 6, 7, 8, 9]",[4]
1,2001-2005,"[1, 3, 4, 5]","[1, 3, 4, 5, 6, 7, 8, 9]",[2]
2,2006-2010,"[3, 5]","[3, 5, 6, 7, 8, 9]","[1, 2, 4]"
3,2011-2015,[5],"[5, 6, 7, 8, 9]","[1, 2, 3, 4]"
4,2016-2020,[],"[6, 7, 8, 9]","[1, 2, 3, 4, 5]"
5,2021-2025,[5],"[5, 6, 7, 8, 9]","[1, 2, 3, 4]"
